In [1]:
import pynoddy
import importlib
importlib.reload(pynoddy)
import pynoddy.history
import pynoddy.output
import pandas as pd
import numpy as np
from exh_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
samples = pd.read_csv('../bregenz model/bregenz_data.csv', delimiter = ',')

In [3]:
history = '../Lechtal model/bregenz_samples.his'
output_name = 'outputs/noddy_out'
pynoddy.compute_model(history, output_name, 
                      noddy_path = r'C:\Users\Sofia\pynoddy\noddyapp\noddy_win64.exe')

hist = pynoddy.history.NoddyHistory(history)
out = pynoddy.output.NoddyOutput(output_name)

#make it hd 
hist.change_cube_size(100)
hist_hd = 'outputs/hist_hd.his'
out_hd = 'outputs/out_hd'
hist.write_history(hist_hd)
pynoddy.compute_model(hist_hd, out_hd, noddy_path = r'C:\Users\Sofia\pynoddy\noddyapp\noddy_win64.exe')
out_hd = pynoddy.output.NoddyOutput(out_hd)

outputs/noddy_out
outputs/out_hd


## FUNCTIONS

In [4]:
def calc_new_position(hist, diff, og_depths, samples):
    samples_noddy_pos = []
    for i in np.arange(11,21):
        p,_,out = ExtractCoords(hist, lith = [i], res = 1)
        t = p[...,2].min()
        
        z = (t*1000) / 3681.39
        
        samples_noddy_pos.append(z)
        
    proposed_exhumation = [x - y - z for x,y,z in zip(samples_noddy_pos, diff, og_depths)]
    samples['exhumation'] = proposed_exhumation
    
    return samples, samples_noddy_pos 

In [5]:
def disturb_property(PH_local, event_list, prop_list, std_list):
    data = []
    for i in event_list:
        event_data = [i]
        for j, prop in enumerate(prop_list):
            new_param = disturb_value(PH_local.events[i], prop_list[j], std_list[j])
            event_data.append(new_param)
            
        data.append(event_data)
    col = ['event_name'] + prop_list
    df = pd.DataFrame(data, columns = col)
    
    return data, df

## CALCULATE ORIGINAL EXHUMATION

In [6]:
event_list = [20,21,22,24] #choose which events to modify
prop_list = ['Slip','Amplitude'] #choose which properties to modify
std_list = [600,200] 

In [7]:
og_depths = []
for event_name, event in hist.events.items():
    if isinstance(event, pynoddy.events.Plug):
        z = event.properties['Z']
        
        og_depths.append(z)

samples_z = []
for i in range(len(samples)):
    z = samples.iloc[i]['Z']
    
    samples_z.append(z)
    
_,samples_noddy_pos = calc_new_position(hist, og_depths, og_depths, samples)   
diff = [x - y for x, y in zip(samples_noddy_pos, samples_z)]
current_exhumation = [x - y - z for x,y,z in zip(samples_noddy_pos, diff, og_depths)]
samples['exhumation'] = current_exhumation

outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out
outputs/temp_out


In [8]:
current_exhumation = samples
samples['respected'] = 0

In [17]:
og_params = []
for i in event_list:
    event_data = [i]
    for j, prop in enumerate(prop_list):
        propert = hist.events[i].properties[prop]
        event_data.append(propert)
    og_params.append(event_data)
    
col = ['event_name'] + prop_list
og_params_df = pd.DataFrame(data, columns = col)

## MCMC SIMULATION

In [23]:
og_params_df

,event_name,Slip,Amplitude
0,20,-10000.0,3064.0
1,21,-9200.0,4528.0
2,22,-9000.0,6792.0
3,24,-12000.0,5000.0


In [22]:
denise = og_params_df
denise['min'] = [-11000, -10200, -10000,-13000]
denise['max'] = [3564]

In [20]:
n_draws = 20
accepted =0
current_params = og_params

In [15]:
for i in range(n_draws):
    
    hist_copy = copy.deepcopy(hist)
    proposed_params, proposed_params_df = disturb_property(hist_copy, prop, std_list)
    
    proposed_exhumation,_ = calc_new_position(hist_copy, diff, og_depths, samples)
    
    #calculate likelihood
    current_likelihood,current_score,current_samples = likelihood_and_score(current_exhumation)
    proposed_likelihood,proposed_score,proposed_samples = likelihood_and_score(proposed_exhumation)
    current_prior = prior_dist(og_params, current_params, std_list)
    proposed_prior = prior_dist(og_params, proposed_params, std_list)
    
    print(f"accepted model of score {proposed_score}")
    acceptance_ratio = (proposed_prior * proposed_likelihood) / (current_prior * current_likelihood)
    print(f"acceptance ratio: {acceptance_ratio}")
    
    random_n = np.random.rand(1)
    print(f"random threshold: {random_n}")
    if acceptance_ratio > random_n:
        #accept
        current_params = proposed_params
        current_exhumation = proposed_exhumation
        
        accepted +=1
    
        #otherwise reject
    

outputs/temp_out


KeyboardInterrupt: 

# FUNCTIONS (TO BE DELETED LATER)

In [6]:
def create_pdf(mean, std_dev):
    def pdf(x):
        coeff = 1 / (std_dev * np.sqrt(2 * np.pi))
        exponent = - ((x - mean) ** 2) / (2 * std_dev ** 2)
        return coeff * np.exp(exponent)
    return pdf

def prior_dist(og_params,proposed_params,std_list):
    prior_prob = 1.0
    for i in range(len(og_params)):
        for j in range(len(std_list)):
            pdf = create_pdf(og_params[i][j+1], std_list[j])
            prior_prob *= pdf(proposed_params[i][j+1])
    return prior_prob

In [28]:
def likelihood_and_score(samples_df):
    
    likelihood = 1.0
    model_score = 0
    
    for i in range(len(samples_df)):
        if samples_df.iloc[i]['group'] in ['a']:
            if samples_df.iloc[i]['exhumation'] < 3000: #non reset AFT sample (B60, always accepted) strict
                likelihood *= 2
                model_score += 1
                samples_df.loc[i,'respected'] += 1
                
            else:
                proximity = (samples_df.iloc[i]['exhumation'] - 3000) / 3000
                rf = np.exp(-proximity)
                likelihood *= rf
                
        
        elif samples_df.iloc[i]['group'] in ['b']:
            if samples_df.iloc[i]['exhumation'] > 4800: #reset AFT sample (B10, never accepted) not strict
                likelihood *= 2
                model_score += 1
                samples_df.loc[i,'respected'] += 1
            else:
                proximity = (4800 - samples_df.iloc[i]['exhumation']) / 4800
                rf = np.exp(-proximity)
                likelihood *= rf
                
        elif samples_df.iloc[i]['group'] in ['c']: #this should be a strict criteria #reset AHe, partially reset AFT
            if samples_df.iloc[i]['exhumation'] > 3200 and samples_df.iloc[i]['exhumation'] < 4800:
                likelihood *= 4
                model_score += 1
                samples_df.loc[i,'respected'] += 1
            else:
                likelihood *= 0.05
                
        elif samples_df.iloc[i]['group'] in ['d']: #reset AHe samples
            if samples_df.iloc[i]['exhumation'] > 3200:
                likelihood *= 2
                model_score += 1
                samples_df.loc[i,'respected'] += 1
            else:
                proximity = (3200 - samples_df.iloc[i]['exhumation']) / 3200
                rf = np.exp(-proximity)
                likelihood *= rf
    return likelihood, model_score, samples_df
            
            

In [29]:
_,_,s = likelihood_and_score(current_exhumation)

In [30]:
s

,X,Y,Z,sample,group,exhumation,respected
0,1626.7528,4990.4679,710,60,a,0.000000,1
1,8145.5806,1315.1883,519,55,c,17.025996,0
2,13815.5822,12609.0000,541,50,d,2973.948008,0
3,15737.3562,14061.0000,521,45,d,4390.000000,1
4,18309.8314,15806.0000,636,40,d,2991.025996,0
5,19436.3479,16453.0000,740,35,d,6042.935010,1
6,21076.8867,17473.0000,765,30,a,6041.961006,0
7,23018.3551,18905.0000,894,25,c,4545.000000,1
8,23685.2056,19362.0000,920,15,d,7871.948008,1
9,25491.5708,792.0000,920,10,b,7771.974004,1
